In [1]:
import time
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import os
import pandas as pd
import locale

In [2]:
url = "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables.asp"

intentos_maximos = 10
intentos = 0
while intentos < intentos_maximos:
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        td_element = soup.find('td', string='Tipo de Cambio Minorista ($ por US$) Comunicación B 9791 - Promedio vendedor')
        if td_element:
            td_siblings = td_element.find_next_siblings('td')
            value = td_siblings[1].text.strip()
            value = value.split('/')[0].strip().replace(',', '.')
            dolar = float(value)
            print(f"Valor del dólar: {dolar}")
            print(f"Se completó el proceso en {intentos+1} intentos")
            break
    except:
        pass
    intentos += 1
    time.sleep(4)

if intentos == intentos_maximos:
    print("No se pudo completar el proceso después de 10 intentos")

Valor del dólar: 267.88
Se completó el proceso en 1 intentos


In [3]:
hojas_descarte = ['¿Quienes somos']

def procesar_archivo_excel(nombre_archivo, hojas_descarte):
    # Leer el archivo Excel
    xls = pd.ExcelFile(nombre_archivo)

    # Obtener las hojas del archivo
    hojas_dexa = [hoja for hoja in xls.sheet_names if hoja != hojas_descarte]

    # Leer cada hoja y almacenarla en un diccionario de DataFrames
    dfs_dexa = {}
    for hoja in hojas_dexa:
        dfs_dexa[hoja] = pd.read_excel(xls, sheet_name=hoja)

    return dfs_dexa

def agregar_columna_proveedor(df, proveedor):
    df.insert(0, "proveedor", proveedor)
    return df

def calcular_usd_final(df, columna, columna_iva, posicion):
    df.insert(posicion, 'USD Final', df[columna] * (1 + df[columna_iva] / 100))
    return df

def calcular_ars_neto(df, columna, posicion):
    df.insert(posicion, 'ARS neto', df[columna] * dolar)
    return df

def calcular_ars_final(df, columna, posicion):
    df.insert(posicion, 'ARS final', df[columna] * dolar)
    return df

In [4]:
ruta_rawdata = Path.cwd() / 'rawdata'

# Cambiar al directorio rawdata
ruta_rawdata = ruta_rawdata.resolve()
os.chdir(ruta_rawdata)

In [5]:
os.listdir()

['dexa.xlsx', 'Lista de precios - GREMIO.xlsx', 'sawerin.xlsx', 'ts.xlsx']

In [6]:
dfs_dexa = procesar_archivo_excel('dexa.xlsx', hojas_descarte)

In [7]:
#DEXA!

In [8]:
for hoja, df in dfs_dexa.items():
    dfs_dexa[hoja] = df.iloc[3:]

# Concatenar todas las hojas en un solo DataFrame
dexa_final = pd.concat(dfs_dexa.values())
# Obtener la primera fila como encabezado
encabezado = dexa_final.iloc[0]

# Asignar la primera fila como nombres de columnas
dexa_final.columns = encabezado

dexa_final = dexa_final.drop(['Imagen Ref.', 'Código'], axis=1)
dexa_final = dexa_final.dropna()
agregar_columna_proveedor(dexa_final, "Dexa")
dexa_final = dexa_final.drop(dexa_final[dexa_final['Stock'] == 'Stock'].index)
dexa_final['Stock'] = dexa_final['Stock'].replace({'Alto': 'Stock Alto', 'Consultar': 'Consultar Stock'})
nuevos_nombres = {'Marca': 'Fabricante', 'Precio': 'usd neto', 'Stock': 'Observacion'}
dexa_final = dexa_final.rename(columns=nuevos_nombres)
columnas_ordenadas = ['proveedor', 'Fabricante', 'Modelo', 'Descripción','IVA %', 'usd neto', 'Observacion']
dexa_final = dexa_final[columnas_ordenadas]
dexa_final["IVA %"] = dexa_final["IVA %"].astype(float)
#pasar valores de 'usd neto' a float (fue medio problematico, despues hay que documentar bien)
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
dexa_final["usd neto"] = dexa_final["usd neto"].apply(lambda x: locale.atof(x) if isinstance(x, str) else x)
calcular_usd_final(dexa_final, 'usd neto', 'IVA %',6)
calcular_ars_neto(dexa_final,'usd neto',7)
calcular_ars_final(dexa_final,'USD Final',8)
dexa_final = dexa_final.round(2)

In [9]:
dfs_sawerin = procesar_archivo_excel('sawerin.xlsx', hojas_descarte)

In [11]:
for hoja, df in dfs_sawerin.items():
    dfs_sawerin[hoja] = df.iloc[8:]

In [12]:
# Concatenar todas las hojas en un solo DataFrame
sawerin_final = pd.concat(dfs_sawerin.values())

In [14]:
# Obtener la primera fila como encabezado
encabezado = sawerin_final.iloc[0]

# Asignar la primera fila como nombres de columnas
sawerin_final.columns = encabezado

In [20]:
sawerin_final = sawerin_final.drop(['Mas Datos', 'Cod.'], axis=1)
agregar_columna_proveedor(sawerin_final, "Sawerin")
columnas = ['Descripcion', 'IVA', 'U$S + IVA', 'U$S IVA inc', 'Stock']

# Eliminar las filas donde las celdas de las columnas especificadas sean NaN
sawerin_final = sawerin_final.dropna(subset=columnas, how='all')

In [35]:
sawerin_final

8,proveedor,Modelo,Descripcion,IVA,U$S + IVA,U$S IVA inc,Stock
8,Sawerin,Modelo,Descripcion,IVA,U$S + IVA,U$S IVA inc,Stock
10,Sawerin,CubeG-5ac60ad,CubeG-5ac60ad,10.50 %,U$S 381.62,U$S 421.69,0
12,Sawerin,K-79,RB5009 rackmount kit K-79,21.00 %,U$S 18.18,U$S 22.00,28
14,Sawerin,18POW,"Fuente 24V 0,8A",21.00 %,U$S 31.25,U$S 37.81,0
15,Sawerin,24HPOW,Fuente 24V 2.5A,21.00 %,U$S 64.06,U$S 77.51,0
...,...,...,...,...,...,...,...
12,Sawerin,HG8247H5,HUAWEI EchoLife HG8247H5 CATV,10.50 %,U$S 159.28,U$S 176.00,1510
14,Sawerin,NaN,UPS2000G 3KVA Single phase in,10.50 %,U$S 674.21,U$S 745.00,0
15,Sawerin,UPS2000G 1KVA,UPS2000G 1KVA Single phase in,10.50 %,U$S 411.76,U$S 455.00,46
16,Sawerin,UPS2000G 2KVA,UPS2000G 2KVA Single phase in,10.50 %,U$S 671.49,U$S 742.00,0
